In [ ]:
import os
import sys

import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from stable_baselines3 import PPO

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import our custom modules
from trading.agents.training_agent_lstm import FXTradingTrainer
from trading.environments.forex_env2_flat import ForexTradingEnv as FX_ENV_FLAT
from sb3_contrib import RecurrentPPO

In [ ]:
from data_management.preprocessor import DataPreprocessor

processor = DataPreprocessor()
not_norm_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/not_norm'
norm_path = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/norm/'

pairs = [f for f in os.listdir(not_norm_path) if f.endswith(".parquet") and not f.startswith(".")]
pairs
for pair in pairs:
    
    pair_name = pair.split('_')[0] + '_' + pair.split('_')[1]
    
    df = pd.read_parquet(f'{not_norm_path}/{pair}')
    df_norm = processor.normalize_simple(df)
    df_norm.to_parquet(f'{not_norm_path}/{pair_name}_5min_1D_norm_10dec.parquet')

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from trading.environments.forex_env_flat_multi_pair import MultipairForexTradingEnv

from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from sb3_contrib import RecurrentPPO
# Load all your currency pair data
pair_data = {
    'AUD_USD': pd.read_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/df_with_all_indics_unbiased/AUD_USD_5min_1D_not_norm_10dec.parquet'),
    'GBP_USD': pd.read_parquet('path_to_gbpusd.parquet'),
    # ... load other pairs
}

sequence = 5
saving_path = f'/Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/notebooks/no_lookahead_bias/'
os.makedirs(saving_path, exist_ok=True)

def make_train_env():
    env = MultipairForexTradingEnv(
        pair_data=pair_data,
        sequence_length=sequence

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(

        df=val_df,
        pair=eur_usd,
        sequence_length=sequence,
        # resample_interval='1h'
    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=50_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}sequence_{sequence}_1day_A2C_indic_full/',
)

model.learn(
    total_timesteps=10_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}{sequence}_best_model.zip')
train_env.save(f'{saving_path}{sequence}_vec_normalize.pkl')




In [ ]:
df = pd.read_parquet('/Users/floriankockler/Library/CloudStorage/OneDrive-kockler/coding_datasets/EUR_USD_5min_1D_all_indic_norm_unbiased.parquet')
len(df)

In [ ]:
# Import our custom modules
from trading.agents.training_agent_lstm import FXTradingTrainer
from trading.environments.forex_env2_flat import ForexTradingEnv as FX_ENV_FLAT
from sb3_contrib import RecurrentPPO

trainer = FXTradingTrainer(
    env_class=FX_ENV_FLAT,
    data_path='./EUR_USD_5min_1D_all_indic_norm_unbiased.parquet',
    pair='EUR_USD',
    sequence_length=5,
    
    model_class=PPO,
    use_sequences=False,
    tensorboard_log="logs/optuna_ppo_08dec_new_data_v2"
)

# Run hyperparameter optimization
best_params = trainer.optimize(
    total_timesteps=1_000_000,  # Steps per trial
    n_trials=40,             # Number of trials to run
    n_startup_trials=10,     # Number of random trials before TPE
    n_evaluations=2,         # Episodes per evaluation
    eval_freq=50_000,          # How often to evaluate
    studyname="ppo_08dec_new_data_v2",
)

# Train final model with best parameters
model = trainer.train(
    total_timesteps=1_000_000,
    eval_freq=10000,
    save_freq=10000,
    hyperparams=best_params
)

In [ ]:
trainer = FXTradingTrainer(
    env_class=FX_ENV_FLAT,
    data_path='./EUR_USD_5min_1D_norm_unbiased_full.parquet',
    pair='EUR_USD',
    sequence_length=5,
    model_class=PPO,
    use_sequences=False,
    tensorboard_log="logs/optuna_ppo_08dec"
)

# Run hyperparameter optimization
best_params = trainer.optimize(
    total_timesteps=500_000,  # Steps per trial
    n_trials=50,             # Number of trials to run
    n_startup_trials=10,     # Number of random trials before TPE
    n_evaluations=2,         # Episodes per evaluation
    eval_freq=50_000          # How often to evaluate
)

# Train final model with best parameters
model = trainer.train(
    total_timesteps=1_000_000,
    eval_freq=10000,
    save_freq=10000,
    hyperparams=best_params
)

In [ ]:
import pandas as pd
df = pd.read_parquet('EUR_USD_5min_1D_norm_unbiased_full.parquet')


BEST RESULTS SO FAR

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from sb3_contrib import RecurrentPPO



all_tickers = [
                             'GBP_CHF', 'GBP_JPY', 'EUR_CHF', 'EUR_JPY', 'USD_CHF',
 
        
        ]

sequence_length =[ 5,
                  ]

eur_usd = 'EUR_USD'

for sequence in sequence_length:
    train_set = f'./EUR_USD_5min_1D_indic_not_norm_unbiased_full.parquet'
    df = pd.read_parquet(train_set)
    dataset_manager = DatasetManager()
    train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

    saving_path = f'/Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/notebooks/no_lookahead_bias/'
    os.makedirs(saving_path, exist_ok=True)

    def make_train_env():
        env = ForexTradingEnv(
            df=train_df,
            pair=eur_usd,
            sequence_length=sequence,

        )
        env = Monitor(env)
        env = DummyVecEnv([lambda: env])
        env = VecNormalize(env, norm_obs=True, norm_reward=True)
        return env

    def make_eval_env():
        env = ForexTradingEnv(

            df=val_df,
            pair=eur_usd,
            sequence_length=sequence,
            # resample_interval='1h'
        )
        env = Monitor(env)
        env = DummyVecEnv([lambda: env])
        env = VecNormalize(env, norm_obs=True, norm_reward=False)
        env.training = False
        return env

    train_env = make_train_env()
    eval_env = make_eval_env()
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=saving_path,
        log_path=saving_path,
        eval_freq=50_000,  # Adjust as needed
        n_eval_episodes=5,
        deterministic=True,
        render=False
    )

    model = PPO(
        'MlpPolicy',
        train_env,
        verbose=0,
        tensorboard_log=f'{saving_path}sequence_{sequence}_1day_A2C_indic_full/',
    )

    model.learn(
        total_timesteps=2_000_000,  # Adjust as needed
        callback=eval_callback
    )

    model.save(f'{saving_path}{sequence}_best_model.zip')
    train_env.save(f'{saving_path}{sequence}_vec_normalize.pkl')

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager



all_tickers = [
                             'GBP_CHF', 'GBP_JPY', 'EUR_CHF', 'EUR_JPY', 'USD_CHF',
 
        
        ]

sequence_length =[ 5,
                  ]

eur_usd = 'EUR_USD'

for sequence in sequence_length:
    train_set = f'./EUR_USD_5min_1D_indic_not_norm_unbiased_full.parquet'
    df = pd.read_parquet(train_set)
    dataset_manager = DatasetManager()
    train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

    saving_path = f'/Users/floriankockler/Code/GitHub.nosync/ai6-gcp-bot/forex_trading_system/notebooks/no_lookahead_bias/'
    os.makedirs(saving_path, exist_ok=True)

    def make_train_env():
        env = ForexTradingEnv(
            df=train_df,
            pair=eur_usd,
            sequence_length=sequence,

        )
        env = Monitor(env)
        env = DummyVecEnv([lambda: env])
        env = VecNormalize(env, norm_obs=True, norm_reward=True)
        return env

    def make_eval_env():
        env = ForexTradingEnv(

            df=val_df,
            pair=eur_usd,
            sequence_length=sequence,
            # resample_interval='1h'
        )
        env = Monitor(env)
        env = DummyVecEnv([lambda: env])
        env = VecNormalize(env, norm_obs=True, norm_reward=False)
        env.training = False
        return env

    train_env = make_train_env()
    eval_env = make_eval_env()
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=saving_path,
        log_path=saving_path,
        eval_freq=50_000,  # Adjust as needed
        n_eval_episodes=5,
        deterministic=True,
        render=False
    )

    model = PPO(
        'MlpPolicy',
        train_env,
        verbose=0,
        tensorboard_log=f'{saving_path}sequence_{sequence}_1day_A2C_indic_full/',
    )

    model.learn(
        total_timesteps=2_000_000,  # Adjust as needed
        callback=eval_callback
    )

    model.save(f'{saving_path}{sequence}_best_model.zip')
    train_env.save(f'{saving_path}{sequence}_vec_normalize.pkl')